# Рекомендательная система на библиотеке surprise 2

Сайт библиотеки: http://surpriselib.com/

Установки библиотеки:
```
$ pip install scikit-surprise```

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from surprise import Dataset

In [ ]:
data = Dataset.load_builtin('ml-100k')

In [ ]:
from surprise import KNNBaseline
from surprise.model_selection import train_test_split

In [ ]:
trainset, testset = train_test_split(data, test_size=.30) #разоббем выборку на обучение и тест

In [ ]:
algo = KNNBaseline(k=20, verbose=False)
predictions = algo.fit(trainset).test(testset)

In [ ]:
df_ratings=pd.DataFrame(columns=['userID', 'itemID', 'rating', 'real_rating'])
for i in predictions:
    #if int(i.uid)==833:
    df_ratings = df_ratings.append({'userID':i.uid, 'itemID': i.iid, 'rating': i.est, 'real_rating': i.r_ui}, ignore_index=True)

In [ ]:
user456=df_ratings[df_ratings.userID=='456']

In [ ]:
threshold=5 # выбираем порог при котором элементы считаем релевантными
n=7
rel_n = set(user456[user456.real_rating>=threshold]['itemID'])

In [ ]:
user456.sort_values(by='rating', ascending=False)[0:n]

,userID,itemID,rating,real_rating
5505,456,474,4.528689,5.0
14558,456,86,4.149827,2.0
14419,456,427,4.119658,4.0
14603,456,197,4.104260,4.0
28162,456,175,4.099644,3.0
7696,456,14,4.036253,5.0
15920,456,174,4.027757,4.0


In [ ]:
t=user456.sort_values(by='rating', ascending=False)[0:n]
ret_n=set(t['itemID'])

In [ ]:
len(rel_n & ret_n)/n

0.2857142857142857

In [ ]:
def Precision_at_n(df_ratings, n=3, threshold=5):
    rel_n = set(df_ratings[df_ratings.real_rating>=threshold]['itemID'])
    t=df_ratings.sort_values(by='rating', ascending=False)[0:n]
    ret_n=set(t['itemID'])
    return len(rel_n & ret_n)/n

In [ ]:
def Avg_Precision_at_n(df_ratings, n=10,threshold=5):
    avg_p=0
    rel_n = set(df_ratings[df_ratings.real_rating>=threshold]['itemID'])
    
    for i in range(1,n+1):
        t=df_ratings.sort_values(by='rating', ascending=False)[0:i]
        ret_n=set(t['itemID'])
        t2=df_ratings.sort_values(by='rating', ascending=False)[i-1:i]
        ret_n2=set(t2['itemID'])
        #print(ret_n2,ret_n)
        if len((ret_n2 & rel_n))>0: #rel(i)
            avg_p=avg_p+len(rel_n & ret_n)/n
    return avg_p/n

In [ ]:
for i in range(3,15):
    print(i,Precision_at_n(user456,i))

3 0.3333333333333333
4 0.25
5 0.2
6 0.3333333333333333
7 0.2857142857142857
8 0.375
9 0.3333333333333333
10 0.3
11 0.2727272727272727
12 0.25
13 0.23076923076923078
14 0.21428571428571427


In [ ]:
for i in range(3,15):
    print(i,Avg_Precision_at_n(user456,i))

3 0.1111111111111111
4 0.0625
5 0.04
6 0.08333333333333333
7 0.061224489795918366
8 0.09375
9 0.07407407407407407
10 0.06000000000000001
11 0.04958677685950413
12 0.041666666666666664
13 0.03550295857988166
14 0.030612244897959183
